In [1]:
# General libraries
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
import joblib

# Utils functions
from utils.utils import kfold, read_datasets

In [2]:
x_train, x_test, y_train, y_test = read_datasets(
    'x_train.csv',
    'x_test.csv',
    'y_train.csv',
    'y_test.csv'
)

In [3]:
folds = kfold()

In [4]:
# Specify range of hyperparameters to tune
hyper_params = {
    'hidden_layer_sizes': [(100,), (80,), (110,)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.002, 0.005],
    'power_t': [0.5],
    'max_iter': [200],
    'shuffle': [True],
    'momentum': [0.5, 0.8, 0.9, 1],
    'early_stopping': [False]

}



# Call GridSearchCV()
model_cv = RandomizedSearchCV(
    estimator = MLPRegressor(),
    param_distributions = hyper_params,
    n_iter=200,
    scoring= 'r2',
    cv = folds,
    verbose = 2,
    return_train_score=True,
    n_jobs = -1,
    refit = True
    )


# Fit the model
best_model = model_cv.fit(x_train, np.ravel(y_train)) 

print(model_cv.best_params_)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


/Users/renan/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [ 1.06293710e-01 -5.27946609e+00  3.66237020e-02  9.04007754e-02
  1.36077095e-02 -1.68767011e-01  1.02198418e-01  1.06011311e-01
  9.33390117e-02 -1.98195375e-02 -8.42564787e-02  1.08160960e-01
  1.21668935e-01  1.41220890e-01  1.06275851e-01 -2.65822696e+00
 -4.39446395e-01 -1.88078208e-01  6.46252779e-02  1.05832506e-01
 -1.27142262e-01  1.04340139e-01  1.81874736e-02  9.20220716e-02
 -3.21079505e-02  1.63230477e-02  9.11084415e-02  9.75895468e-02
 -4.47816443e-01 -9.33261468e-01 -3.58091821e-01 -3.74516371e-02
 -2.40515326e-01  9.73874539e-02  6.48047081e-02 -2.07169542e-01
  9.72725719e-02             nan  1.18673753e-01  1.16230029e-01
 -3.25683102e+00 -1.54923600e-01  9.46910694e-02  1.31472986e-01
 -5.13111597e-01 -2.40102605e-01 -1.93798721e-01             nan
 -8.71552299e-01  1.03654938e-01  9.29072966e-02  1.22935007e-0

{'solver': 'sgd', 'shuffle': True, 'power_t': 0.5, 'momentum': 0.9, 'max_iter': 200, 'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (110,), 'early_stopping': False, 'alpha': 0.0001, 'activation': 'tanh'}


/Users/renan/Library/Python/3.7/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [5]:
# Create new model with best_params_ from grid search
# Use cross validation on the best_params_ model

mlp_best = MLPRegressor(
    hidden_layer_sizes=model_cv.best_params_['hidden_layer_sizes'],
    activation=model_cv.best_params_['activation'],
    solver=model_cv.best_params_['solver'],
    alpha=model_cv.best_params_['alpha'],
    learning_rate=model_cv.best_params_['learning_rate'],
    learning_rate_init=model_cv.best_params_['learning_rate_init'],
    power_t=model_cv.best_params_['power_t'],
    max_iter=model_cv.best_params_['max_iter'],
    shuffle=model_cv.best_params_['shuffle'],
    momentum=model_cv.best_params_['momentum'],
    early_stopping=model_cv.best_params_['early_stopping'],
    )


mlp_best.fit(x_train, y_train)

/Users/renan/Library/Python/3.7/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/renan/Library/Python/3.7/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPRegressor(activation='tanh', hidden_layer_sizes=(110,),
             learning_rate='adaptive', solver='sgd')

In [6]:
mlp_best.predict(x_test)

array([52.78234243, 51.85333624, 40.50676087, 39.65287434, 70.87047149,
       50.61452904, 60.2627015 , 48.12503759, 57.33702632, 66.36852643,
       54.92141756, 57.68531831, 87.63234689, 66.40618723, 82.91559553,
       30.28363047, 78.88711468, 50.93698578, 69.32490502, 55.26829159,
       46.30059082, 49.96023852, 76.5447495 , 68.27622282, 50.45836345,
       77.1003379 , 54.27321588, 43.59828317, 88.28809727, 39.40197083,
       58.29652871, 48.47483042, 47.92556534, 53.06488   , 48.64109433,
       47.26122508, 68.00383635, 87.34602934, 65.1894306 , 70.45175361,
       43.91588422, 66.33389224, 66.0250955 , 68.8737904 , 43.45559072,
       70.24033755, 85.83828484, 60.07747141, 41.47201569, 30.47775224,
       51.63564234, 69.67358265, 48.28624669, 61.51693564, 65.04677597,
       66.3101646 , 52.59339976, 63.71378157, 53.62720113, 81.35145423,
       58.09560758, 61.50581148, 61.8762005 , 88.3688526 , 64.5571993 ,
       56.48373726, 42.71122914, 77.23653468, 40.3528721 , 52.82

In [7]:
mlp_best.score(x_test, y_test)

0.20223824883521002

In [8]:
filename = '../models/mlpRegressor.joblib'
joblib.dump(mlp_best, filename)

['../models/mlpRegressor.joblib']